This noteboook is designed as a prototype script for grabbing articles from the JSON file.

First let's create a small batch of the JSON objects.

In [2]:
pathway = "/Volumes/Seagate Backup Plus Drive 1/RC_2015-01"

In [16]:
import json
import shelve
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
stopwords.append('[deleted]')

In [17]:
print stopwords

[u'i', u'me', u'my', u'myself', u'we', u'our', u'ours', u'ourselves', u'you', u'your', u'yours', u'yourself', u'yourselves', u'he', u'him', u'his', u'himself', u'she', u'her', u'hers', u'herself', u'it', u'its', u'itself', u'they', u'them', u'their', u'theirs', u'themselves', u'what', u'which', u'who', u'whom', u'this', u'that', u'these', u'those', u'am', u'is', u'are', u'was', u'were', u'be', u'been', u'being', u'have', u'has', u'had', u'having', u'do', u'does', u'did', u'doing', u'a', u'an', u'the', u'and', u'but', u'if', u'or', u'because', u'as', u'until', u'while', u'of', u'at', u'by', u'for', u'with', u'about', u'against', u'between', u'into', u'through', u'during', u'before', u'after', u'above', u'below', u'to', u'from', u'up', u'down', u'in', u'out', u'on', u'off', u'over', u'under', u'again', u'further', u'then', u'once', u'here', u'there', u'when', u'where', u'why', u'how', u'all', u'any', u'both', u'each', u'few', u'more', u'most', u'other', u'some', u'such', u'no', u'nor', u

In [34]:
def json_to_db(db_name,num_of_comments):
    

    '''
    Goes through the txt file, grabs the JSON objects in it.
    '''
    # open database
    d = shelve.open(db_name)
    
    # Set counter
    count = 0
    
    # Iterate through N JSON objects in file
    with open(pathway) as myfile:
        for item in myfile:
            if count < num_of_comments:
                
                # Load the JSON object
                json_object = json.loads(item)
                
                # Now append additional text to database
                if d.has_key(json_object['subreddit']):
                    
                    temp = d[json_object['subreddit']]      # extracts the copy
                    temp = temp+' '+json_object['body']     # mutates the copy
                    d[json_object['subreddit']] = temp      # stores the copy right back, to persist it

                # Or set the first entry for that subreddit
                else:
                    d[json_object['subreddit']] = json_object['body']
                
    
                count += 1
            else:
                d.close()
                break

In [35]:
json_to_db('10000_comments',10000)

In [36]:
d = shelve.open('10000_comments.db')

In [45]:
d['MakeupAddiction']

u"Thanks! Now on to NYE makeup! [deleted] Right!? My face loves me, my wallet hates me.  Simple and Tasteful. :) Dipbrow is definitely something that has to be learned haha. I just barely stick the end of my brush into the pot and thats usually enough to cover my whole brow. But I also use the brow wiz first to outline my brows.  They look great on you, especially the red. How much do these cost if you don't mind me asking? Yeah she did that in high school...I've also recently read that she lied about being molested. Not sure how I feel about her now especially since she hardly does beauty vids anymore. She def is lacking when it comes to effort in her vids. I mixed a little of a Tarte stain (berry colored) with the baby lips clear chapstick, I've had the Tarte stain forever and the label is worn off so i dont know the exact color but i'll try and find out! and thank you!! Seriously. The recommendations for Gerard cosmetics need to chill the fuck out. Sona Gasparian's no makeup-makeup 

In [46]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')
tokenizer.tokenize(d['MakeupAddiction'])

[u'Thanks',
 u'Now',
 u'on',
 u'to',
 u'NYE',
 u'makeup',
 u'deleted',
 u'Right',
 u'My',
 u'face',
 u'loves',
 u'me',
 u'my',
 u'wallet',
 u'hates',
 u'me',
 u'Simple',
 u'and',
 u'Tasteful',
 u'Dipbrow',
 u'is',
 u'definitely',
 u'something',
 u'that',
 u'has',
 u'to',
 u'be',
 u'learned',
 u'haha',
 u'I',
 u'just',
 u'barely',
 u'stick',
 u'the',
 u'end',
 u'of',
 u'my',
 u'brush',
 u'into',
 u'the',
 u'pot',
 u'and',
 u'thats',
 u'usually',
 u'enough',
 u'to',
 u'cover',
 u'my',
 u'whole',
 u'brow',
 u'But',
 u'I',
 u'also',
 u'use',
 u'the',
 u'brow',
 u'wiz',
 u'first',
 u'to',
 u'outline',
 u'my',
 u'brows',
 u'They',
 u'look',
 u'great',
 u'on',
 u'you',
 u'especially',
 u'the',
 u'red',
 u'How',
 u'much',
 u'do',
 u'these',
 u'cost',
 u'if',
 u'you',
 u'don',
 u't',
 u'mind',
 u'me',
 u'asking',
 u'Yeah',
 u'she',
 u'did',
 u'that',
 u'in',
 u'high',
 u'school',
 u'I',
 u've',
 u'also',
 u'recently',
 u'read',
 u'that',
 u'she',
 u'lied',
 u'about',
 u'being',
 u'molested',
 u

In [49]:
# Example dictionary
d = {'string1' : 1, 'string2' : 2, 'string3' : 3}

from pymongo import Connection
conn = Connection()
db = conn['example-database']
collection = db['example-collection']

for string, num in d.items():
    collection.save({'_id' : string, 'value' : num})
# testing
newD = {}
for obj in collection.find():
    newD[obj['_id']] = obj['value']
print newD
# output is: {u'string2': 2, u'string3': 3, u'string1': 1}

{u'string2': 2, u'string3': 3, u'string1': 1}
